# Infer MITOCHONDRIA - part 5️⃣

--------------

## OBJECTIVE: ✅ Infer sub-cellular component  MITOCHONDRIA  in order to understand interactome 



Dependencies:
The MITOCHONDRIA  inference rely on the CYTOSOL, which is SOMA&~NUCLEI.  



## IMPORTS

In [1]:
# top level imports
from pathlib import Path
import os, sys
from collections import defaultdict

import numpy as np
import scipy

# TODO:  prune the imports.. this is the big set for almost all organelles
# # function for core algorithm
from scipy import ndimage as ndi
import aicssegmentation
from aicssegmentation.core.seg_dot import dot_3d_wrapper, dot_slice_by_slice, dot_2d_slice_by_slice_wrapper, dot_3d
from aicssegmentation.core.pre_processing_utils import ( intensity_normalization, 
                                                         image_smoothing_gaussian_3d,  
                                                         image_smoothing_gaussian_slice_by_slice )
from aicssegmentation.core.utils import topology_preserving_thinning, size_filter
from aicssegmentation.core.MO_threshold import MO
from aicssegmentation.core.utils import hole_filling
from aicssegmentation.core.vessel import filament_2d_wrapper, vesselnessSliceBySlice
from aicssegmentation.core.output_utils import   save_segmentation,  generate_segmentation_contour
                                                 
from skimage import filters
from skimage.segmentation import watershed
from skimage.feature import peak_local_max
from skimage.morphology import remove_small_objects, binary_closing, ball , dilation, remove_small_holes   # function for post-processing (size filter)
from skimage.measure import label

# # package for io 
from aicsimageio import AICSImage

import napari

### import local python functions in ../infer_subc_2d
sys.path.append(os.path.abspath((os.path.join(os.getcwd(), '..'))))

from infer_subc_2d.utils.file_io import (read_input_image, 
                                                                    list_image_files, 
                                                                    export_ome_tiff, 
                                                                    etree_to_dict, 
                                                                    save_parameters, 
                                                                    load_parameters, 
                                                                    export_ndarray)
from infer_subc_2d.utils.img import *
from infer_subc_2d.organelles.nuclei import infer_NUCLEI
from infer_subc_2d.organelles.soma import infer_SOMA
from infer_subc_2d.organelles.cytosol import infer_CYTOSOL

%load_ext autoreload
%autoreload 2

test_img_n = 5

# IMAGE PROCESSING Objective 5:  infer MITOCHONDRIA


## summary of steps (Workflow #1 & #2)

INPUT
- channel  3
- CY mask

PRE-PROCESSING
-  smoothe / remove noise

CORE-PROCESSING
-  segment objects

- POST-PROCESSING
  - filter objects

OUTPUT
- object MITOCHONDRIA 



------------------------
# LOAD RAW IMAGE DATA
Identify path to _raw_ image data and load our example image


In [2]:
# build the datapath
# all the imaging data goes here.
data_root_path = Path(os.path.expanduser("~")) / "Projects/Imaging/data"

# linearly unmixed ".czi" files are here
data_path = data_root_path / "raw"
im_type = ".czi"

# get the list of all files
img_file_list = list_image_files(data_path,im_type)
test_img_name = img_file_list[test_img_n]

bioim_image = read_input_image(test_img_name)
img_data = bioim_image.image
raw_meta_data = bioim_image.raw_meta
ome_types = []
meta_dict = bioim_image.meta

# get some top-level info about the RAW data
channel_names = meta_dict['name']
img = meta_dict['metadata']['aicsimage']
scale = meta_dict['scale']
channel_axis = meta_dict['channel_axis']

/opt/anaconda3/envs/napariNEW/lib/python3.9/site-packages/ome_types/_convenience.py:105: FutureWarning: The default XML parser will be changing from 'xmlschema' to 'lxml' in version 0.4.0.  To silence this warning, please provide the `parser` argument, specifying either 'lxml' (to opt into the new behavior), or'xmlschema' (to retain the old behavior).
  d = to_dict(os.fspath(xml), parser=parser, validate=validate)


## Get default parameters, including  "optimal" Z

takes ~ 4 seconds to calculate

In [3]:
load_Z_from_params = False


if load_Z_from_params:

    default_params = load_parameters( test_img_name.split("/")[-1], data_root_path / "intermediate" )

    ch_to_agg = default_params["ch_to_agg"]
    nuc_ch = default_params['nuc_ch']
    optimal_Z = default_params["optimal_Z"] #find_optimal_Z(img_data, nuc_ch, ch_to_agg) 
else:
    ch_to_agg = (1,2,3,4,5,6)
    nuc_ch = 0
    optimal_Z = find_optimal_Z(img_data, nuc_ch, ch_to_agg) 

    default_params = defaultdict(str, **{
        #"intensity_norm_param" : [0.5, 15]
        "intensity_norm_param" : [0],
        "gaussian_smoothing_sigma" : 1.34,
        "gaussian_smoothing_truncate_range" : 3.0,
        "dot_2d_sigma" : 2,
        "dot_2d_sigma_extra" : 1,
        "dot_2d_cutoff" : 0.025,
        "min_area" : 10,
        "low_level_min_size" :  100,
        "median_filter_size" : 4,
        "ch_to_agg" : (1,2,3,4,5,6), # exclude residual
        "nuc_ch" : 0,
        "optimal_Z": optimal_Z,
    })
    save_parameters(default_params, test_img_name.split("/")[-1], data_root_path / "intermediate" )
# make sure we have removed Z
if len(scale)>2:
    scale = scale[1:]


img_2D = img_data[:,[optimal_Z],:,:].copy()


## get the inferred nuclei object

(takes < 1 sec)

In [4]:


raw_nuclei = img_2D[0].copy()
NU_object, NU_label, out_p =  infer_NUCLEI(raw_nuclei.copy(), default_params) 


intensity normalization: min-max normalization with NO absoluteintensity upper bound


## get the inferred soma object

(takes < 1 sec)

In [5]:
raw_soma = (4. * img_2D[1].copy() + 
                               1. * img_2D[5].copy() + 
                               1. * img_2D[7].copy() )

SO_object, SO_label, out_p =  infer_SOMA(raw_soma.copy(), NU_label, default_params) 
CY_object =  infer_CYTOSOL(SO_object, NU_object) 


intensity normalization: min-max normalization with NO absoluteintensity upper bound
intensity normalization: min-max normalization with NO absoluteintensity upper bound
intensity normalization: min-max normalization with NO absoluteintensity upper bound


# WORKFLOW #1 

Generally following the Allen Cell Segmenter procedure, but doing more aggressive contrast scaling than their prescribed contrast scaling.

> Using Allen Cell Segmenter Tom20as a potential Mitochondrian segmenter to start with  start from [Allen Cell](https://www.allencell.org/cell-observations/category/mitochondria).   using [seg_tomm20.py]("../../../../aics-segmentation/aicssegmentation/structure_wrapper/seg_tomm20.py")

## summary of steps

INPUT
- channel  2
- CY mask

PRE-PROCESSING
-  smoothe / remove noise

CORE-PROCESSING
-  'vesselness' enhancement
-  threshold objects

- POST-PROCESSING
  - filter objects

OUTPUT
- object MITOCHONDRIA 


### INPUT

In [6]:

###################
# INPUT
###################
raw_mito    = img_2D[2].copy()

# mask object
masked_mito = apply_mask(raw_mito,CY_object)


### PRE-PROCESSING

In [7]:
###################
# PRE_PROCESSING
###################

intensity_norm_param = [0, 9] # from Allen Cell Segmenter LAMP1  workflow

# Linear-ish smoothing
struct_img = intensity_normalization( masked_mito ,  scaling_param=intensity_norm_param)

med_filter_size = 3  

gaussian_smoothing_sigma = 1.3
gaussian_smoothing_truncate_range = 3.0

struct_img = median_filter(struct_img,    size=med_filter_size  )

structure_img_smooth = image_smoothing_gaussian_slice_by_slice(   struct_img,
                                                                                                                        sigma=gaussian_smoothing_sigma,
                                                                                                                        truncate_range=gaussian_smoothing_truncate_range,
                                                                                                                    )


# log_img, d = log_transform( structure_img_smooth ) 
# struct_img = intensity_normalization(  log_img  ,  scaling_param=[0] )  

struct_img = structure_img_smooth

### CORE PROCESSING

In [8]:
###################
# CORE_PROCESSING
###################
# dot and filiment enhancement - 2D


################################
## PARAMETERS for this step ##
vesselness_sigma = [1.5]
vesselness_cutoff = 0.1
# 2d vesselness slice by slice
response = vesselnessSliceBySlice(structure_img_smooth, sigmas=vesselness_sigma, tau=1, whiteonblack=True)
bw = response > vesselness_cutoff


# # from Sec61b workflows - playground references Tom20
# f2_param = [[1.5, 0.16]]
# bw = filament_2d_wrapper(structure_img_smooth, f2_param)

### POST-PROCESSING

In [9]:

# # 3D
###################
# POST_PROCESSING
###################

################################
## PARAMETERS for this step ##

small_object_max = 3

MT_object = size_filter_2D(bw, 
                                                min_size= small_object_max**2, 
                                                connectivity=1)


#### Visualize with `napari`
Visualize the first-pass segmentation and labeling with `napari`.

In [10]:


viewer = napari.view_image(
    raw_mito,
    scale=scale
)


viewer.scale_bar.visible = True
viewer.add_image(
    structure_img_smooth,
    scale=scale
)


<Image layer 'structure_img_smooth' at 0x14e0ede50>

In [11]:

viewer.add_image(
    bw,
    scale=scale
)

<Image layer 'bw' at 0x17920c850>

### DEFINE `infer_MITOCHONDRIA` function

In [12]:
##########################
#  infer_MITOCHONDRIA
##########################
def _infer_MITOCHONDRIA(struct_img, CY_object,  in_params) -> tuple:
    """
    Procedure to infer MITOCHONDRIA  from linearly unmixed input.

    Parameters:
    ------------
    struct_img: np.ndarray
        a 3d image containing the MITOCHONDRIA signal

    CY_object: np.ndarray boolean
        a 3d image containing the NU labels

    in_params: dict
        holds the needed parameters

    Returns:
    -------------
    tuple of:
        object
            mask defined boundaries of MITOCHONDRIA
        parameters: dict
            updated parameters in case any needed were missing
    """
    out_p= in_params.copy()


    struct_img = apply_mask(struct_img,CY_object)
    
    ###################
    # PRE_PROCESSING
    ###################                         
    scaling_param =  [0,9]   
    struct_img = intensity_normalization(struct_img, scaling_param=scaling_param)
    out_p["intensity_norm_param"] = scaling_param

   # make a copy for post-post processing
    scaled_signal = struct_img.copy()

    med_filter_size = 3   
    structure_img = median_filter(struct_img,    size=med_filter_size  )
    out_p["median_filter_size"] = med_filter_size 

    gaussian_smoothing_sigma = 1.34
    gaussian_smoothing_truncate_range = 3.0
    struct_img = image_smoothing_gaussian_slice_by_slice(   struct_img,
                                                                                                        sigma=gaussian_smoothing_sigma,
                                                                                                        truncate_range = gaussian_smoothing_truncate_range
                                                                                                    )
    out_p["gaussian_smoothing_sigma"] = gaussian_smoothing_sigma 
    out_p["gaussian_smoothing_truncate_range"] = gaussian_smoothing_truncate_range

    # log_img, d = log_transform( struct_img ) 
    # struct_img = intensity_normalization( log_img ,  scaling_param=[0] )  
    # struct_img = intensity_normalization( struct_img ,  scaling_param=[0] )  


   ###################
    # CORE_PROCESSING
    ###################
    ################################
    ## PARAMETERS for this step ##
    vesselness_sigma = [1.5]
    vesselness_cutoff = 0.1
    # 2d vesselness slice by slice
    response = vesselnessSliceBySlice(structure_img_smooth, sigmas=vesselness_sigma, tau=1, whiteonblack=True)
    bw = response > vesselness_cutoff

    out_p["vesselness_sigma"] = vesselness_sigma 
    out_p["vesselness_cutoff"] = vesselness_cutoff 


    ###################
    # POST_PROCESSING
    ###################

    # MT_object = remove_small_objects(bw > 0, min_size=small_object_max**2, connectivity=1, in_place=False)
    small_object_max = 3
    struct_obj = size_filter_2D(bw, # wrapper to remove_small_objects which can do slice by slice
                                                            min_size= small_object_max**2, 
                                                            connectivity=1)
    out_p['small_object_max'] = small_object_max

    retval = (struct_obj,  out_p)
    return retval


------------------

# TEST `infer_MITOCHONDRIA` function

In [13]:

###################
# INPUT
###################
raw_mito    = img_2D[2].copy()

MT_object, out_p =  _infer_MITOCHONDRIA(raw_mito, CY_object, default_params) 

In [14]:

viewer.add_image(
    MT_object,
    scale=scale
)
viewer.add_labels(
    label(MT_object),
    scale=scale
)

<Labels layer 'Labels' at 0x18347cb20>

In [15]:
from infer_subc_2d.organelles.mitochondria import infer_MITOCHONDRIA


img_2D = img_data[:,[optimal_Z],:,:].copy()
raw_mito    = img_2D[2].copy()

MT_object, out_p =  infer_MITOCHONDRIA(raw_mito, CY_object, default_params) 

---------------------

 🚧 WIP 🚧 (🚨🚨🚨🚨 )

# WORKFLOW #2 (WIP)
as per 6/22 CellProfiler pipeline from MCZ
 > ADAPTIVE... not sure exactly how to impliment this...

  >ReduceNoise performs non-local means noise reduction. Instead of only using a neighborhood of pixels around a central pixel for denoising, such as in GaussianFilter, multiple neighborhoods are pooled together. The neighborhood pool is determined by scanning the image for regions similar to the area around the central pixel using a correlation metric and a cutoff value.
   
## summary of steps (Workflow #1 & #2)

INPUT
- ch 3
- CY mask

PRE-PROCESSING
-  non-local noise reduction
  - size:4, distance:2, cut-off:0.1

CORE-PROCESSING
  - adaptive Otsu
    - diameter: (2,200)
  - two classes
    - threshold smoothing scale: 0
    - threshold correction factor: .1
    - threshold bounds: (0.14 1)
    - adaptive window: 20 pixels

POST-PROCESSING
  - N/A





### INPUT

In [24]:

###################
# INPUT
###################
raw_mito    = img_2D[2].copy()

# mask object
masked_mito = apply_mask(raw_mito,CY_object)


### PRE-PROCESSING

In [26]:
###################
# PRE_PROCESSING
###################

intensity_norm_param = [0] # 

# Linear-ish smoothing
raw_mito = intensity_normalization( masked_mito ,  scaling_param=intensity_norm_param)

med_filter_size =3  

gaussian_smoothing_sigma = 1.3
gaussian_smoothing_truncate_range = 3.0

struct_img = median_filter(raw_mito,    size=med_filter_size  )

structure_img_smooth = image_smoothing_gaussian_slice_by_slice(   struct_img,
                                                                                                                        sigma=gaussian_smoothing_sigma,
                                                                                                                        truncate_range=gaussian_smoothing_truncate_range,
                                                                                                                    )


# log_img, d = log_transform( structure_img_smooth ) 
# struct_img = intensity_normalization(  log_img  ,  scaling_param=[0] )  

struct_img = structure_img_smooth

intensity normalization: min-max normalization with NO absoluteintensity upper bound


### CORE PROCESSING

In [27]:
###################
# CORE_PROCESSING
###################
# enhance spreckles - 40
big_struct_rad = 40
big_img = enhance_speckles(struct_img.copy(),big_struct_rad, True)

#adaptive_otsu(big_struct) # three class - middle foreground
# adaptive window- 20
#size: 10,100
# threshold smooth 1.34
# threshold correction 1
# threshold (0.1497,1)
#fill holes
# "adatpive" thresholds are NOT currently working... will use      
# "Masked Object Thresholding" - 3D
low_level_min_size = 10
bw_big, _bw_low_level = MO(big_img, 
                                                global_thresh_method='ave', 
                                                object_minArea=low_level_min_size, 
                                                extra_criteria=True,
                                                local_adjust= 0.5, 
                                                return_object=True,
                                                dilate=True)

In [28]:

# enhance speckles
#   adaptive_sauvola(sm_struct) 
# adaptive window- 10
#size: 2,10
# threshold smooth 1.34
# threshold correction 1
# threshold (0.05,1)
#fill holes

# enhance spreckles - 10
sm_struct_rad = 20
sm_img = enhance_speckles(struct_img.copy(),sm_struct_rad, True)

# "adatpive" thresholds are NOT currently working... will use      
# "Masked Object Thresholding" - 3D
low_level_min_size = 2
bw_sm, _bw_low_level = MO(sm_img, 
                                                global_thresh_method='ave', 
                                                object_minArea=low_level_min_size, 
                                                extra_criteria=True,
                                                local_adjust= 0.5, 
                                                return_object=True,
                                                dilate=True)
 # or this?                                               
#    struct_obj = struct_img > filters.threshold_otsu(struct_img)
#     threshold_value_log = threshold_otsu_log(struct_img)
#     threshold_factor = 0.9 #from cellProfiler
#     thresh_min = .1
#     thresh_max = 1.
#     threshold = min( max(threshold_value_log*threshold_factor, thresh_min), thresh_max)
#     struct_obj = struct_img > threshold



### POST-PROCESSING

In [29]:
###################
# POST_PROCESSING
###################


# 3D
# cleaned_img = remove_small_objects(removed_holes>0, 
#                                                             min_size=minArea, 
#                                                             connectivity=1, 
#                                                             in_place=False)
small_object_max = 10
cleaned_img_big = size_filter(bw_big, # wrapper to remove_small_objects which can do slice by slice
                                                         min_size= small_object_max**2,
                                                         method = "slice_by_slice",
                                                         connectivity=1)


#                                                             in_place=False)
small_object_max = 2
cleaned_img_sm = size_filter(bw_sm, # wrapper to remove_small_objects which can do slice by slice
                                                         min_size= small_object_max**2,
                                                         method = "slice_by_slice",
                                                         connectivity=1)



cleaned_img = np.logical_or(cleaned_img_big, cleaned_img_sm)



#### Visualize with `napari`
Visualize the first-pass segmentation and labeling with `napari`.

In [30]:


viewer.add_image(
    cleaned_img,
    scale=scale
)

viewer.scale_bar.visible = True
viewer.add_image(
    cleaned_img_big,
    scale=scale
)

viewer.add_image(
    big_img,
    scale=scale
)
viewer.add_image(
    cleaned_img_big,
    scale=scale
)
viewer.add_image(
    sm_img,
    scale=scale
)

<Image layer 'sm_img' at 0x17ac9dfa0>

In [ ]:
# save updated parameters for ongoing testing
save_parameters(default_params, test_img_name.split("/")[-1], data_root_path / "intermediate" )

'/Users/ahenrie/Projects/Imaging/data/intermediate/ZSTACK_PBTOhNGN2hiPSCs_BR3_N14_Unmixed.czi_params.pkl'



# DEFINE `infer_MITOCHONDRIA_CP` function (WIP)

In [38]:
# copy this to base.py for easy import
# mangle so we can call from base.py
def _infer_MITOCHONDRIA_CP(struct_img: np.ndarray,  CY_object:np.array,  in_params:dict) -> tuple:
    """
    Procedure to infer MITOCHONDRIA from linearly unmixed input as per CellProfiler procedure

    Parameters:
    ------------
    struct_img: np.ndarray
        a 3d image containing the MITOCHONDRIA signal (should be CY masked)
    CY_object: np.ndarray boolean
        a 3d image containing the NU labels

    in_params: dict
        holds the needed parameters

    Returns:
    -------------
    tuple of:
        object
            mask defined boundaries of SOMA
    
        parameters: dict
            updated parameters in case any needed were missing
    
    """
    out_p= in_params.copy()

    ###################
    # PRE_PROCESSING
    ###################                         
    #TODO: replace params below with the input params
    scaling_param =  [0]   
    struct_img = intensity_normalization(struct_img, scaling_param=scaling_param)
    out_p["intensity_norm_param"] = scaling_param


    # 2D smoothing
    # make a copy for post-post processing
    scaled_signal = struct_img.copy()

    med_filter_size = 9   
    # structure_img_median_3D = ndi.median_filter(struct_img,    size=med_filter_size  )
    struct_img = median_filter_slice_by_slice( 
                                                                    struct_img,
                                                                    size=med_filter_size  )
    out_p["median_filter_size"] = med_filter_size 

    gaussian_smoothing_sigma = 3.
    gaussian_smoothing_truncate_range = 3.0
    struct_img = image_smoothing_gaussian_slice_by_slice(   struct_img,
                                                                                                        sigma=gaussian_smoothing_sigma,
                                                                                                        truncate_range = gaussian_smoothing_truncate_range
                                                                                                    )
    out_p["gaussian_smoothing_sigma"] = gaussian_smoothing_sigma 
    out_p["gaussian_smoothing_truncate_range"] = gaussian_smoothing_truncate_range

    #    edges = filters.scharr(struct_img)
    # struct_img, d = log_transform( struct_img ) 
    # struct_img = intensity_normalization(  struct_img,  scaling_param=[0] )
    ###################
    # CORE_PROCESSING
    ###################
    # enhance spreckles - 40
    big_struct_rad = 40
    big_img = enhance_speckles(struct_img,big_struct_rad)
    out_p["big_struct_rad"] = big_struct_rad 

    local_adjust = 0.5
    low_level_min_size = 10
    bw_big, _bw_low_level = MO(big_img, 
                                                global_thresh_method='ave', 
                                                object_minArea=low_level_min_size **2 , 
                                                extra_criteria=True,
                                                local_adjust= local_adjust, 
                                                return_object=True,
                                                dilate=True)
    out_p["local_adjust"] = local_adjust 
    out_p["low_level_min_size_big"] = low_level_min_size_big 

    # enhance spreckles - 10
    sm_struct_rad = 20
    sm_img = enhance_speckles(struct_img,sm_struct_rad)
    out_p["big_struct_rad"] = big_struct_rad 

    # "adatpive" thresholds are NOT currently working... will use      
    # "Masked Object Thresholding" - 3D
    low_level_min_size_sm = 2
    bw_sm, _bw_low_level = MO(sm_img, 
                                                    global_thresh_method='ave', 
                                                    object_minArea=low_level_min_size_sm **2, 
                                                    extra_criteria=True,
                                                    local_adjust= local_adjust, 
                                                    return_object=True,
                                                    dilate=True)

    out_p["sm_struct_rad"] = sm_struct_rad 
    out_p["low_level_min_size_sm"] = low_level_min_size_sm 

    ###################
    # POST_PROCESSING
    ###################

    # 3D
    small_object_max_big = 10
    bw_big = size_filter(bw_big, # wrapper to remove_small_objects which can do slice by slice
                                                            min_size= small_object_max_big**2,
                                                            method = "slice_by_slice",
                                                            connectivity=1)


    #                                                             in_place=False)
    small_object_max_sm = 3
    bw_sm = size_filter(bw_sm, # wrapper to remove_small_objects which can do slice by slice
                                                            min_size= small_object_max_sm**2,
                                                            method = "slice_by_slice",
                                                            connectivity=1)
    out_p['small_object_max_sm'] = small_object_max

    struct_obj = np.logical_or(bw_big, bw_sm)

    retval = (struct_obj, out_p)
    return retval

[autoreload of infer_subc_2d.organelles.lysosomes failed: Traceback (most recent call last):
  File "/opt/anaconda3/envs/napariNEW/lib/python3.9/site-packages/IPython/extensions/autoreload.py", line 257, in check
    superreload(m, reload, self.old_objects)
  File "/opt/anaconda3/envs/napariNEW/lib/python3.9/site-packages/IPython/extensions/autoreload.py", line 455, in superreload
    module = reload(module)
  File "/opt/anaconda3/envs/napariNEW/lib/python3.9/importlib/__init__.py", line 159, in reload
    raise ImportError(msg.format(parent_name),
ImportError: parent 'infer_subc_2d.organelles' not in sys.modules
]
